<a href="https://colab.research.google.com/github/rajaamani/Zocket_Task/blob/main/Zocket_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers wandb
!pip install tensorflow tensorflow_datasets


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import wandb
import tensorflow_datasets as tfds
import mss


In [ ]:
# Load the dataset
dataset = tfds.load("tiny_shakespeare", split="train")

In [ ]:
# Extract text from the dataset
raw_text = "\n".join([example["text"].numpy().decode("utf-8") for example in dataset])


In [ ]:
# Tokenize the text
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenized_text = tokenizer(raw_text, return_tensors="pt", max_length=1024, truncation=True, padding=True)


In [ ]:
# Save tokenized data to a file
with open("tokenized_data.txt", "w") as file:
    for seq in tokenized_text['input_ids']:
        file.write(" ".join(map(str, seq.tolist())) + "\n")

In [ ]:
# Create TextDataset and DataCollatorForLanguageModeling
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="tokenized_data.txt",
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
# Fine-tune LORA Adapters
model = GPT2LMHeadModel.from_pretrained('gpt2')
config = GPT2Config.from_pretrained('gpt2')


In [ ]:
# Define Trainer and TrainingArguments
training_args = TrainingArguments(
    output_dir="./LORA_fine_tuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_total_limit=2,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)


In [ ]:
# Fine-tune the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=18, training_loss=5.190171135796441, metrics={'train_runtime': 2.13, 'train_samples_per_second': 15.493, 'train_steps_per_second': 8.451, 'total_flos': 2155659264000.0, 'train_loss': 5.190171135796441, 'epoch': 3.0})

In [ ]:
# Incorporate WandB for Logging
wandb.init(project="shakespeare_tuning")
wandb.config.update(training_args)

eval/loss,▁▁
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
eval_loss,▁
train/epoch,▁▁▁
train/global_step,▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


In [ ]:
# Evaluate and log metrics
eval_results = trainer.evaluate(eval_dataset=train_dataset)
wandb.log({"eval_loss": eval_results["eval_loss"]})

In [ ]:
# Calculate perplexity from the evaluation loss
perplexity = torch.exp(torch.tensor(eval_results["eval_loss"]))

# Log perplexity to WandB
wandb.log({"perplexity": perplexity.item()})